In [3]:
import os
import ccxt
# import config
import schedule
import pandas as pd
pd.set_option('display.max_rows', None)
from datetime import datetime
import time
import pytz
import json

# import warnings
# warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, './strategy')
from importlib import reload  

In [8]:
odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string', 'strategy': 'string'})
ondf = odf[odf.orderId.isnull()]
# pd.isna(odf['orderId'])
ondf

,date_entered,exchange,pair,strategy,order_params,orderId
id,,,,,,
10,2021-08-19 14:45:04.560470,kucoin,DOT/USDT,BuyLevelOrder,"{""trigger_price"": 25.26, ""qty"": 0.02}",<NA>
11,2021-08-19 14:45:05.560470,binance,BTC/USDT,BuyLevelOrder,"{""trigger_price"": 40000, ""qty"": 0.001}",<NA>
2,2021-08-19 14:45:06.560470,ftx,DOT/USDT,BuyLevelOrder,"{""trigger_price"": 25, ""qty"": 0.01}",<NA>


In [3]:
odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string'})
ondf = odf[odf.status=='new']
for index, orow in ondf.iterrows():
    module = __import__(orow.strategy)
    reload(module)

In [4]:
exchanges = {}
odf = ondf = xnodf = False
def run_bot():
    global odf, ondf, xnodf
    global exchanges
    config = {}
    with open ('config.json') as js: 
        config = json.load(js)

    for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
            excred = exconf['cred']
            for crk, crv in excred.items():
                if crv[0] == "$" : 
                    excred[crk] = os.getenv(crv[1:])

            exchanges[exkey] = getattr(ccxt, exkey) (excred)
    
    print(f'{datetime.now()} Checking for new orders...')
    odf = pd.read_csv ('order-data/orders.csv', index_col='id', dtype={'id': 'string', 'orderId': 'string'})
    ondf = odf[odf.orderId.isnull()]

    for exkey, exchange in exchanges.items():
        xnodf = ondf[ondf.exchange == exkey]
        symbols =  xnodf.symbol.unique()
        tickers = exchange.fetchTickers (symbols)

        for oind, orow in xnodf.iterrows():
            ticker = tickers[orow.symbol]
            orobj = eval(orow.order, {"__builtins__": {}}, {'ticker': ticker})
            if orobj['trigger']:
                # pass
                print(f'{datetime.now()} Executing order {oind} ...')
                pass
                order = exchange.create_order(orow.symbol, orobj['type'], orobj['side'], orobj['amount'], orobj['price'], orobj['params'])
                if order['id']:
                    if orow.succ_order:
                        succorobj = eval(orow.succ_order, {"__builtins__": {}}, {'ticker': ticker, 'order': order})
                        succorder = exchange.create_order(orow.symbol, succorobj['type'], succorobj['side'], succorobj['amount'], succorobj['price'], succorobj['params'])
                        # print(succorder)
                        print(f'    Executing successive order ...')
                    
                odf.loc[oind, "orderId"] = order['id']
                odf.to_csv('order-data/orders.csv', index=True)


                

            # module = __import__(orow.strategy)
            # order_class = getattr(module, orow.strategy)
    #     order_params = json.loads(orow.order_params)
    #     order_instance = order_class(exchange, orow.pair, tickers[orow.pair], order_params)
            # order = order_instance.execute({})
    #     if order:
    #         odf.loc[oind, "orderId"] = order['id']
    #         odf.to_csv('order-data/orders.csv', index=True)


# schedule.every(5).seconds.do(run_bot)
# while True:
#     schedule.run_pending()
#     time.sleep(1)
run_bot()


2021-08-20 17:02:09.652225 Checking for new orders...
2021-08-20 17:02:11.298231 Executing order 10 ...
    Executing successive order ...


In [7]:
xnodf

,date_entered,exchange,symbol,strategy,order,succ_order,orderId
id,,,,,,,
10,2021-08-19 14:45:04.560470,kucoin,DOT/USDT,BuyLevelOrder,"{'trigger': ticker['ask'] <= 27, 'type': 'mark...","{'type': 'market', 'amount': order['amount'], ...",<NA>


In [38]:
orow = ondf.iloc[0]
odf.index[odf.id == orow.id][0]

0

AttributeError: 'Series' object has no attribute 'id'

In [11]:
order_instance.execute()

order params: {'trigger_price': 25, 'qty': 0.01}


In [17]:
ondf

,exchange,pair,status,strategy,order_params
date,,,,,
2021-08-19 14:45:04.560470,KUCOIN,DOT/USDT,new,BuyLevelOrder,"{""trigger_price"": 25, ""qty"": 0.01}"
2021-08-19 14:45:04.560470,KUCOIN,BTC/USDT,new,BuyLevelOrder,"{""trigger_price"": 40000, ""qty"": 0.001}"


In [44]:
odf

,exchange,pair,status,strategy,params
date,,,,,
2021-08-19 14:45:04.560470,KUCOIN,DOT/USDT,new,BuyLevel,{}


In [20]:
time.localtime()

time.struct_time(tm_year=2021, tm_mon=8, tm_mday=19, tm_hour=14, tm_min=42, tm_sec=3, tm_wday=3, tm_yday=231, tm_isdst=1)